In [2]:
import pandas as pd

In [3]:
# create a subset of data from the full dataset (only run once)
# full = open('full_dataset.tsv', 'r')
# subset = open('recipe_dataset.tsv', 'w')
# for _ in range(100001):
#     subset.write(full.readline())
# full.close()
# subset.close()

In [4]:
# open raw data file
recipe_df = pd.read_csv('recipe_dataset.tsv', sep='\t')
recipe_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
recipe_df.head()

,id,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [31]:
# making set of all ingredients
all_ingredients = set()
for entry in recipe_df['NER']:
    all_ingredients.update([e.lower() for e in entry[2:-2].split('\", \"')])

In [32]:
ingredients = open('ingredient_list.txt', 'w')
for i in all_ingredients:
    ingredients.write(f'{i}\n')
ingredients.close()

In [33]:
# print all ingredients
print(len(all_ingredients))
# print(all_ingredients)

15049


In [7]:
# create sets of ingredients for each dietary restriction checker
vegan = set(['gelatin', 'jello', 'honey', 'egg', 'cheese', 'milk', 'chocolate', 'marshmallow', 'yogurt', 'cream', 'butter'])
vegetarian = set(['pork', 'chicken', 'beef', 'bacon', 'fish', 'tuna', 'ham', 'gizzard', 'sirloin', 'veal', 'scallops', 'sausage', 'burger', 'crayfish', 'hen'])
lactose = set(['cheese', 'milk', 'yogurt'])
gluten = set(['bread', 'pie', 'cake', 'cereal', 'noodle', 'pasta', 'croutons', 'cracker', 'cookie', 'gravy', 'dressing', 'soup', 'tofu', 'sauce'])
halal = set(['pork', 'bacon', 'ham', 'alcohol', 'beer', 'whiskey', 'vodka', 'rum', 'wine', 'gin', 'sake', 'tequila', 'bourbon'])
kosher = set()